In [12]:
# data lib
import pandas as pd 
import numpy as np

# json lib
import json

#path lib
import os

#google lib
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [13]:
# setting env
path = os.getcwd()
path_data = path + '/data/'
path_connnector = path + '/connector/'

filename_order = 'TR_OrderDetails'
filename_product = 'TR_Products'
filename_property = 'TR_PropertyInfo'

In [14]:
# load dateset
data_order = pd.read_csv(path_data + f"{filename_order}.csv")
data_product = pd.read_csv(path_data + f"{filename_product}.csv")
data_property = pd.read_csv(path_data + f"{filename_property}.csv")


In [15]:
# data manipulation
data_order['Property'] = data_order['PropertyID'] \
                          .map(data_property.set_index('Prop ID')['PropertyCity'].to_dict())

data_order['Product'] = data_order['ProductID'] \
                          .map(data_product.set_index('ProductID')['ProductName'].to_dict())

data_order['Price'] = data_order['ProductID']\
                         .map(data_product.set_index('ProductID')['Price'].to_dict())

data_order['Sales'] = data_order['Quantity'] * data_order['Price']

cols_order = ['OrderID','OrderDate','Property','Product','Quantity','Price','Sales']
data_order = data_order[cols_order]

data_order['OrderDate'] = pd.to_datetime(data_order['OrderDate']).dt.strftime('%Y-%m-%d')


C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-01-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14-01-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15-01-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-pa

In [16]:
data_order.head()

,OrderID,OrderDate,Property,Product,Quantity,Price,Sales
0,1,2015-01-01,Las Vegas,Office Chair,1,85,85
1,2,2015-01-01,Chicago,Chest of Drawers,2,150,300
2,3,2015-01-01,Kansas City,Hammer,2,17,34
3,4,2015-01-01,Los Angeles,Washcloth,2,3,6
4,5,2015-01-01,Dallas,Computer Desk,3,102,306


In [17]:
data_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   OrderID    5000 non-null   int64 
 1   OrderDate  5000 non-null   object
 2   Property   5000 non-null   object
 3   Product    5000 non-null   object
 4   Quantity   5000 non-null   int64 
 5   Price      5000 non-null   int64 
 6   Sales      5000 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 273.6+ KB


In [18]:
#conn Google API
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(path_connnector + 'zippy-chain-345512-c5021583a18e.json', scope)
client = gspread.authorize(creds)

#conn Google Sheet
file = client.open('data_de6')
worksheet = file.worksheet('Sheet3')


In [66]:
#export data
worksheet.update([data_order.columns.values.tolist()] + data_order.values.tolist())


{'spreadsheetId': '1iCDLpT0rNLUP5bAX3t39VtAGpocwAPqkstuj4YMkEyk',
 'updatedRange': 'Sheet2!A1:G5001',
 'updatedRows': 5001,
 'updatedColumns': 7,
 'updatedCells': 35007}

288
